# Imports

In [ ]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
%matplotlib inline
%reload_ext watermark
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import seaborn as sb
from IPython.display import display
import sklearn as sk
import sklearn.neural_network as sknn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt



# Data

In this section we load both datasets we use, COVID-19 dataset and countries population dataset , clean the data and add some new columns.  

The datasets we use are:

* https://www.kaggle.com/imdevskp/corona-virus-report for the COVID-19 dataset

* https://www.kaggle.com/tanuprabhu/population-by-country-2020 for population information per country


## COVID-19 Dataset

In [ ]:
# Loading datasets

full_table = pd.read_csv('datasets/covid_19_clean_complete.csv', 
                          na_values=['NaN'],
                          parse_dates=['Date'])

# Adding Active cases column
full_table['Active'] = (full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']).apply(lambda x: x if x >= 0 else 0)

# filling missing values
full_table[['Province/State']] = full_table[['Province/State']].fillna('')
full_table[['Confirmed','Deaths','Recovered','Active']] = full_table[['Confirmed','Deaths','Recovered','Active']].fillna(0)

full_table.sample(6)


## Population Dataset


In [ ]:
pop_table = pd.read_csv('datasets/population_by_country_2020.csv',
                        na_values=['N.A.'])


# Selecting only the Country and Population columns
pop_table = pop_table.iloc[:,[0,1,4,9]]



# Renaming columns
pop_table.columns = ['Country/Region', 'Population', 'Population Density (P/Km²)','Urban Population %']

# Most of the entries with urban population as NaN in the population dataset that we are going to use have 100% as of 2020
pop_table[['Urban Population %']] = pop_table[['Urban Population %']].fillna('100 %')
pop_table['Urban Population %'] = pop_table['Urban Population %'].map(lambda x: int(x.split(' ')[0]))


In [ ]:
pop_table.info()
pop_table.isna().sum()

## Removing ship data

The dataset also includes data from the various ships that had COVID19 outbreaks. Since we only need the information per country we removed it from the dataset.

In [ ]:
# ship rows
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Province/State'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MS Zaandam')

# ship
ship = full_table[ship_rows]

# dropping ship rows 
full_table = full_table[~(ship_rows)]

ship.sample(6)


## Fixing country names


### Fixing mismatched names between datasets

Here we manually set the names so that the join between datasets works.


In [ ]:
fix_name_only = {
    'Sao Tome & Principe': 'Sao Tome and Principe',
    "Côte d'Ivoire": "Cote d'Ivoire",
    "United States": "US",
    "Czech Republic (Czechia)": 'Czechia',
    'Myanmar': 'Burma',
    'Taiwan': 'Taiwan*',
    'Saint Kitts & Nevis': 'Saint Kitts and Nevis',
    'Macao' : 'Macau'
}

for original,new in fix_name_only.items():
    full_table.loc[full_table['Country/Region'] == new, 'Country/Region'] = original
    full_table.loc[full_table['Province/State'] == new, 'Province/State'] = original

missing_countries = set(full_table['Country/Region']).difference(set(pop_table['Country/Region']))

# # print(sorted(pop_table['Country/Region'].unique()))
# if len(missing_countries) != 0:
#     print(missing_countries)


### Replacing Country/Region with Province/State

The population dataset has entries for autonomous regions, for example Greenland. Here we rewrite the Country/Region column with the Province/State name so we can easily join the population dataset. For example, Greenland exists in the population dataset so what we do is replace Denmark (the Country column of Greenland) with Greenland.

In [ ]:

province_set = set(full_table['Province/State']).intersection(set(pop_table['Country/Region']))

no_data = set(['Saint Vincent and the Grenadines','Kosovo','Congo','West Bank and Gaza'])

for province in province_set:
    if province in no_data:
        continue
    full_table.loc[ full_table['Province/State'] == province,'Country/Region'] = province 



In [ ]:
# Check for null values
full_table.isna().sum()

## Grouping data

Here we are grouping data by Date and Country so we can add population and cases per million afterwards.

### Group by Country

In [ ]:
full_grouped = full_table.groupby(['Country/Region','Lat','Long','Date'])['Confirmed','Deaths','Recovered','Active'].sum().reset_index()
full_grouped_nolat = full_table.groupby(['Country/Region','Date'])['Confirmed','Deaths','Recovered','Active'].sum().reset_index()

full_grouped.head()

### Adding population
In this section, we merge both datasets by Country/Region.

In [ ]:
full_grouped = pd.merge(full_grouped,pop_table,on=['Country/Region'])
full_grouped_nolat = pd.merge(full_grouped_nolat,pop_table,on=['Country/Region'])

full_grouped

### Calculating new cases per day

To calculate the number of

In [ ]:
# Dataframe with latitude and longitude
temp = full_grouped.groupby(['Country/Region', 'Date'])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()

mask = temp['Country/Region'] != temp['Country/Region'].shift(1)

temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan


temp.columns = ['Country/Region', 'Date','New cases', 'New deaths', 'New recovered']


full_grouped = pd.merge(full_grouped,temp, on=['Country/Region', 'Date'])

full_grouped = full_grouped.fillna(0)

full_grouped[['New cases','New deaths','New recovered']] = full_grouped[['New cases','New deaths','New recovered']].astype('int64')

#############################################################################################################################
# # Dataset with no lat and long

temp = full_grouped_nolat.groupby(['Country/Region', 'Date' ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()

mask = temp['Country/Region'] != temp['Country/Region'].shift(1)

temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']

full_grouped_nolat = pd.merge(full_grouped_nolat,temp, on=['Country/Region', 'Date'])

full_grouped_nolat = full_grouped_nolat.fillna(0)

full_grouped_nolat[['New cases','New deaths','New recovered']] = full_grouped_nolat[['New cases','New deaths','New recovered']].astype('int64')


### World Data

In [ ]:
world_data = full_grouped.groupby(['Date'])['Confirmed','Deaths','Recovered','Active','Population'].sum().reset_index()
world_data.loc[world_data['Date'] == world_data['Date'].max()]
world_data.head()
world_pop_total = world_data['Population'].max()

In [ ]:
# Check information on types and null values
full_grouped.info()
full_grouped.loc[full_grouped['Urban Population %'].isnull()]['Country/Region'].unique()


In [ ]:
full_grouped.sample(6)

### Calculating Cases per Million of People




In [ ]:

def calc_permillion(df):
    df['Confirmed per million'] = round((df['Confirmed'] / df['Population']) * 1000000)
    df['Deaths per million']    = round((df['Deaths'] / df['Population']) * 1000000)
    df['Recovered per million'] = round((df['Recovered'] / df['Population']) * 1000000)
    df['Active per million']    = round((df['Active'] / df['Population']) * 1000000)
    return df

def calc_permillion_world():
    world_data['Confirmed per million'] = round((world_data['Confirmed'] / world_data['Population']) * 1000000)
    world_data['Deaths per million']    = round((world_data['Deaths'] / world_data['Population']) * 1000000)
    world_data['Recovered per million'] = round((world_data['Recovered'] / world_data['Population']) * 1000000)
    world_data['Active per million']    = round((world_data['Active'] / world_data['Population']) * 1000000)



per_million = calc_permillion(full_grouped)
per_million_nolat = calc_permillion(full_grouped_nolat)

per_million.head()

### Value Truncation

Since the COVID-19 dataset only has data from  22nd of January of 2020 onwards, we will define Date from here moving foward as days since the 22nd of January of 2020.

In [ ]:
per_million['Date'].min()

This function does that:

In [ ]:
def daysSinceJan(d):
    return d.toordinal() - datetime(2020,1,20).toordinal()

def revertdaysSince(d):
    return datetime(2020,1,20) + timedelta(days=d)

In [ ]:
per_million_nolat.info()

In [ ]:
per_million['Days Since Jan'] = per_million['Date'].map(daysSinceJan)

per_million_nolat['Days Since Jan'] = per_million['Date'].map(daysSinceJan)

In [ ]:
per_million =  per_million.sort_values(['Date','Country/Region'],ascending=[True, True])
per_million.head()

In [ ]:
per_million_nolat.isna().sum()

### Adding population percentage

In [ ]:
per_million['Pop %'] = per_million['Population'].apply(lambda x: x/world_pop_total * 100)
per_million_nolat['Pop %'] = per_million_nolat['Population'].apply(lambda x: x/world_pop_total * 100)

In [ ]:
full_grouped.head()

In [ ]:
per_million_nolat

In [ ]:
def removeChinaData(df):
    df = df[full_grouped['Country/Region'] != 'China']

removeChinaData(full_grouped)
removeChinaData(full_grouped_nolat)
removeChinaData(per_million_nolat)
removeChinaData(per_million)

In [ ]:
per_million.loc[per_million['Country/Region'] == 'Portugal']

In [ ]:
per_million_nolat.isna().sum()

# Model Training

## Inputs and outputs

This function will return a pair of inputs and outputs given the country:

In [ ]:
def world_train_data(df_full,country=None,out=['Confirmed per million',	'Deaths per million',	'Recovered per million']):
    df = df_full
    input  = ['Date','Population Density (P/Km²)','Urban Population %', 'Pop %']
    if country is not None:
        df = df_full.loc[ df_full['Country/Region'] == country]
        input = ['Date']
    return df[input], df[out]


In [ ]:
def nn_by_country_train(
    country,
    input=['Days Since Jan','Population Density (P/Km²)','Urban Population %','Pop %'],
    out= ['Confirmed per million','Deaths per million','Recovered per million'],
    hidden_layer_sizes=(100,100,100,60,60,60)
):
    df = per_million_nolat.loc[ per_million_nolat['Country/Region'] == country]
    X = df[input].values
    Y = df[out].values
    nn = sknn.MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation='relu',
        solver='adam',
        alpha=0.00001,
        batch_size='auto',
        max_iter=1000,
        n_iter_no_change=100)

    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.30,shuffle=True)
    nn.fit(X_train,y_train) # Train the model
    y_pred = nn.predict(X_test)

    nnr2 = nn.score(X_test,y_test) # Calculate R² for the model
    nnmae = mean_absolute_error(y_test,y_pred) # Mean Absolute Error
    nnmse = mean_squared_error(y_test,y_pred)

    print("R2:",nnr2)
    print("MAE:",nnmae)
    print("MSE:",nnmse)

    return nn

In [ ]:
def print_graph(world_viz,things = ['Confirmed','Deaths','Recovered'],label='C/D/R',x = 'Date',title='Title',limit_x=None):
    sb.set()

    dd = world_viz.melt([x],var_name=label, value_name='Cases',value_vars=things)


    chart = sb.relplot(x=x,y='Cases',hue=label,data=dd,kind='line')

    chart.set(title=title)
    chart.fig.autofmt_xdate()
    if limit_x is not None:
        plt.axvline(limit_x,0,1,linewidth=4, color='r')

    plt.show()#%%

def world_train_data(df_full,country=None,out=['Confirmed per million',	'Deaths per million',	'Recovered per million']):
    df = df_full
    input  = ['Date','Population Density (P/Km²)','Urban Population %', 'Pop %']
    if country is not None:
        df = df_full.loc[ df_full['Country/Region'] == country]
        input = ['Date']
    return df[input], df[out]


In [ ]:
def nn_by_country_train(
    country,
    input=['Days Since Jan','Population Density (P/Km²)','Urban Population %','Pop %'],
    out= ['Confirmed per million','Deaths per million','Recovered per million'],
    hidden_layer_sizes=(100,100,100,60,60,60)
):
    df = per_million_nolat.loc[ per_million_nolat['Country/Region'] == country]
    X = df[input].values
    Y = df[out].values
    nn = sknn.MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation='relu',
        solver='adam',
        alpha=0.00001,
        batch_size='auto',
        max_iter=1000,
        n_iter_no_change=100)

    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.30,shuffle=True)
    nn.fit(X_train,y_train) # Train the model
    y_pred = nn.predict(X_test)

    nnr2 = nn.score(X_test,y_test) # Calculate R² for the model
    nnmae = mean_absolute_error(y_test,y_pred) # Mean Absolute Error
    nnmse = mean_squared_error(y_test,y_pred)

    print("R2:",nnr2)
    print("MAE:",nnmae)
    print("MSE:",nnmse)

    return nn

In [ ]:
def print_graph(world_viz,things = ['Confirmed','Deaths','Recovered'],label='C/D/R',x = 'Date',title='Title',limit_x=None):
    sb.set()

    dd = world_viz.melt([x],var_name=label, value_name='Cases',value_vars=things)


    chart = sb.relplot(x=x,y='Cases',hue=label,data=dd,kind='line')

    chart.set(title=title)
    chart.fig.autofmt_xdate()
    if limit_x is not None:
        plt.axvline(limit_x,0,1,linewidth=1, color='r',linestyle='--')

    plt.show()

In [ ]:
def getPredInp(country):
    return [per_million.loc[per_million['Country/Region'] == country]['Population Density (P/Km²)'].max(),
    per_million.loc[per_million['Country/Region'] == country]['Urban Population %'].max(),
    per_million.loc[full_grouped['Country/Region'] == country]['Pop %'].max()]

# per_million.loc[per_million['Country/Region'] == 'Portugal'].groupby(['Lat','Long']).size()

In [ ]:
def print_predict(model,country,minprev=0,offset=10,maxprev=daysSinceJan(datetime.now().date() - timedelta(days=1))):

    country_pop = per_million_nolat.loc[per_million_nolat['Country/Region'] == country]['Population'].max()
    ip = []

    for dat in range(minprev,maxprev+offset+1): # Predict from 0 to 155 days from January 1st 2020
        ip.append([dat, *getPredInp(country)]) # Hard Coded Pop Density, Urban Pop %, Latitude and Longitude

    out = nn.predict(ip)

    nl = []

    for i,o in zip(ip,out):
        nl.append([*i,*o])

    futurepredict = pd.DataFrame(nl,columns=['Date','Population Density (P/Km²)','Urban Population %',"Pop %",'Confirmed','Deaths','Recovered'])

    futurepredict['Confirmed'] = futurepredict['Confirmed'].map(lambda x: round((x/1000000) * country_pop))
    futurepredict['Recovered'] = futurepredict['Recovered'].map(lambda x: round((x/1000000) * country_pop))
    futurepredict['Deaths'] = futurepredict['Deaths'].map(lambda x: round((x/1000000) * country_pop) )

    futurepredict['Date'] = futurepredict['Date'].map(revertdaysSince) 
    predict = futurepredict.loc[(futurepredict['Date'] >= revertdaysSince(maxprev - 30)) & (futurepredict['Date'] <= revertdaysSince(maxprev + offset))]


    print_graph(per_million_nolat.loc[per_million_nolat['Country/Region'] == country],['Confirmed','Deaths','Recovered'], title='Real Data')
    print_graph(futurepredict,['Confirmed','Deaths','Recovered'],x='Date', title="Future prediction",limit_x=revertdaysSince(maxprev))



In [ ]:
country = 'Mexico'
nn = nn_by_country_train(country)


In [ ]:
print_predict(nn,country,offset=500)